In [6]:
import numpy as np

In [209]:

class Card:
    
    def __init__(self,val,suit):
        if val < 1 or val > 13:
            print(f'value must be between 1 and 13 inclusive! You entered val = {val}')
            raise ValueError
        if suit not in 'HDCS':
            print(f'suit must be H (heart), D (diamond), C (club), or S (spade). You entered suit = {suit}')
            raise ValueError
        self.val = val
        self.suit = suit
        self.color = 'Black'
        if self.suit in 'HD':
            self.color = 'Red'
    
    def __str__(self):
        return f'{self.val}{self.suit}'

class Deck:
    
    _rng = np.random.default_rng(1)
    
    def __init__(self):
        self.cards = [Card(val,suit) for val in range(1,14) for suit in 'HDCS']
        type(self)._rng.shuffle(self.cards)
    
    def __str__(self):
        return str([str(card) for card in self.cards])

    def __len__(self):
        return len(self.cards)

    def __getitem__(self,index):
        return self.cards[index]
    
    def draw(self,n):
        if len(self) == 0:
            print('WARNING! Trying to draw from empty deck')
        drawn = self.cards[:n]
        self.cards = self.cards[n:]
        return drawn

class Pile:
    
    def __init__(self,cards):
        self.showing = [cards[0]] if len(cards) > 0 else []
        self.hidden = cards[1:]
    
    def __str__(self):
        return ' '.join(map(str,self.showing)) + ' || ' + ' '.join(map(str,self.hidden))

    def head(self):
        return self.showing[0]

    def tail(self):
        return self.showing[-1]

    def empty(self):
        return len(self.showing) == 0 and len(self.hidden) == 0

    def canAdd(self,card):
        raise NotImplemented

    def add(self,card):
        raise NotImplemented


class BoardPile(Pile):

    def __init__(self,cards):
        super().__init__(cards)
    
    def canAdd(self,card,player):
        if self.empty():
            return card.val == 13 or player.allKingsOut
        return card.val == self.tail().val-1 and card.color != self.tail().color

    def add(self,card):
        self.showing.append(card)

class AcePile(Pile):

    def __init__(self,cards):
        super().__init__(cards)
    
    def __str__(self):
        if len(self.showing) == 0:
            return '_'
        return str(self.tail())
    
    def canAdd(self,card):
        if len(self.showing) == 0:
            return card.val == 1
        return card.val == self.tail().val + 1 and card.suit == self.tail().suit

    # Assert: len(self.showing) should always be 0 or 1
    def add(self,card):
        if len(self.showing) > 0:
            self.hidden.append(self.showing.pop())
        self.showing.append(card)


def randomStrat(possibleMoves):
    rng = np.random.default_rng(12345)
    moveType = rng.choice(list(possibleMoves.keys()))
    print(f'{moveType=}')
    print(possibleMoves[moveType])
    return rng.choice(possibleMoves[moveType])

def simpleStrat(moveTypePrefs):
    def recommendedMove(possibleMoves):
        for pref in moveTypePrefs:
            if len(possibleMoves[pref]) > 0:
                return possibleMoves[pref][0]
        return None
    return recommendedMove

class Player:
    
    def __init__(self,strat=None):
        self.hand = Deck()
        self.board = [BoardPile(self.hand.draw(i)) for i in range(1,8)]
        self.allKingsOut = False
        if strat is None:
            self.strat = simpleStrat(['p2p','p2a','h2p','h2a'])
        else :
            self.strat = strat

    def show(self):
        for i,pile in enumerate(self.board):
            print(f'Pile {i}: {str(pile)}')
        print('---')
        print('Hand:',self.hand)

    def possibleMoves_pileToPile(self):
        moves = []
        for p1 in self.board:
            for p2 in self.board:
                if p1 == p2:
                    continue
                if p2.canAdd(p1.head(),self):
                    moves.append({'source':p1,'target':p2,'moverFunc':self.move_pileToPile})
        return moves

    def possibleMoves_handToPile(self):
        moves = []
        for card in self.hand[2::3]:
            for pile in self.board:
                if pile.canAdd(card,self):
                    moves.append({'source':card,'target':pile,'moverFunc':self.move_handToPile})
        return moves

    def possibleMoves_pileToAces(self,acePiles):
        moves = []
        for pile in self.board:
            for acePile in acePiles:
                if acePile.canAdd(pile.tail()):
                    moves.append({'source':pile,'target':acePile,'moverFunc':self.move_pileToAces})
        return moves

    def possibleMoves_handToAces(self,acePiles):
        moves = []
        for card in self.hand[2::3]:
            for acePile in acePiles:
                if acePile.canAdd(card):
                    moves.append({'source':card,'target':acePile,'moverFunc':self.move_handToAces})
        return moves

    def possibleMoves(self,acePiles):
        return {
            'p2p':self.possibleMoves_pileToPile(),
            'h2p':self.possibleMoves_handToPile(),
            'p2a':self.possibleMoves_pileToAces(acePiles),
            'h2a':self.possibleMoves_handToAces(acePiles)
        }

    def showPossibleMoves(self,acePiles):
        print('Possible Moves:')
        for moveType,moves in self.possibleMoves(acePiles).items():
            print(moveType)
            for move in moves:
                sourceToShow = move['source'].head() if isinstance(move['source'],Pile) else move['source'] 
                print('source',sourceToShow,'target',move['target'])

    # assert: target.canAdd(source.head()) before running this!
    def move_pileToPile(self,source,target):
        while len(source.showing) > 0:
            target.add(source.showing.pop(0))
        if not source.empty():
            source.add(source.hidden.pop(0))

    # assert target.canAdd(source) before running this!
    def move_handToPile(self,source,target):
        target.add(source)
        self.hand.remove(source)

    # assert target.canAdd(source.showing.head()) before running this!
    def move_pileToAces(self,source,target):
        target.add(source.showing.pop(0))
        if len(source.showing) == 0 and len(source.hidden) != 0:
            source.add(source.hidden.pop(0))

    # assert target.canAdd(source) before running this!
    def move_handToAces(self,source,target):
        target.add(source)
        self.hand.remove(source)


class Game:
    
    def __init__(self,nPlayers):
        self.nPlayers = nPlayers
        self.players = [Player() for _ in range(self.nPlayers)]
        self.acePiles = [AcePile([]) for _ in range(4*self.nPlayers)]
        self.finished = False
    
    def show(self):
        print('===== Aces =====')
        for acePile in self.acePiles:
            print(acePile,end=' ')
        print('\n')
        for i,player in enumerate(self.players):
            print(f'===== Player #{i} =====')
            player.show()
            print('\n')

    def play(self):
        # For 1 player only!! Haven't figured out how want to implement multiple ppl yet
        pass
        p = self.players[0]
        test = ''
        while test == '' and not self.finished:
            test = input('')
            self.show()
            p.showPossibleMoves(self.acePiles)
            move = p.strat(p.possibleMoves(self.acePiles))
            # NOTE: Idea: could have possibleMoves_* return also a moverFunc which tells how to execute the move
            if move is None:
                self.finished = False
            else :
                print('Move Chosen:','source',move['source'],'target',move['target'])
                move['moverFunc'](move['source'],move['target'])
            print('\n\n Next Round \n\n')

# Untested
# move piles
# canAdd with kingsOut implementation


        

g = Game(1)
g = Game(1)
g.play()
# g = Game(1)
# g = Game(1)
# g.show()
# p = g.players[0]
# showMoves(p,g.acePiles)
# moves = p.possibleMoves(g.acePiles)
# m1 = moves['p2p'][0]
# p.movePile(m1['source'],m1['target'])

# print()
# print('New Round!')
# print()

# g.show()
# showMoves(p,g.acePiles)
# moves = p.possibleMoves(g.acePiles)
# m2 = moves['p2p'][-1]
# p.movePile(m2['source'],m2['target'])

# print()
# print('New Round!')
# print()

# g.show()

===== Aces =====
_ _ _ _ 

===== Player #0 =====
Pile 0: 13C || 
Pile 1: 3H || 5H
Pile 2: 12D || 2C 11S
Pile 3: 11H || 2D 11D 13H
Pile 4: 6H || 3C 7H 6C 9C
Pile 5: 5D || 1C 10D 5S 3S 12S
Pile 6: 4C || 7C 10C 9H 7D 12H 2H
---
Hand: ['13D', '4D', '6S', '8C', '8D', '10H', '12C', '1D', '1H', '8H', '8S', '4S', '10S', '3D', '11C', '4H', '9D', '7S', '1S', '5C', '9S', '6D', '2S', '13S']


Possible Moves:
p2p
source 3H target 4C || 7C 10C 9H 7D 12H 2H
source 12D target 13C || 
source 4C target 5D || 1C 10D 5S 3S 12S
h2p
source 4S target 5D || 1C 10D 5S 3S 12S
source 11C target 12D || 2C 11S
p2a
h2a
source 1H target _
source 1H target _
source 1H target _
source 1H target _
Move Chosen: source 3H || 5H target 4C || 7C 10C 9H 7D 12H 2H


 Next Round 




===== Aces =====
_ _ _ _ 

===== Player #0 =====
Pile 0: 13C || 
Pile 1: 5H || 
Pile 2: 12D || 2C 11S
Pile 3: 11H || 2D 11D 13H
Pile 4: 6H || 3C 7H 6C 9C
Pile 5: 5D || 1C 10D 5S 3S 12S
Pile 6: 4C 3H || 7C 10C 9H 7D 12H 2H
---
Hand: ['13D', '4D', '6S', '8C', '8D', '10H', '12C', '1D', '1H', '8H', '8S', '4S', '10S', '3D', '11C', '4H', '9D', '7S', '1S', '5C', '9S', '6D', '2S', '13S']


Possible Moves:
p2p
source 12D target 13C || 
source 4C target 5H || 
source 4C target 5D || 1C 10D 5S 3S 12S
h2p
source 4S target 5H || 
source 4S target 5D || 1C 10D 5S 3S 12S
source 11C target 12D || 2C 11S
p2a
h2a
source 1H target _
source 1H target _
source 1H target _
source 1H target _
Move Chosen: source 12D || 2C 11S target 13C || 


 Next Round 




===== Aces =====
_ _ _ _ 

===== Player #0 =====
Pile 0: 13C 12D || 
Pile 1: 5H || 
Pile 2: 2C || 11S
Pile 3: 11H || 2D 11D 13H
Pile 4: 6H || 3C 7H 6C 9C
Pile 5: 5D || 1C 10D 5S 3S 12S
Pile 6: 4C 3H || 7C 10C 9H 7D 12H 2H
---
Hand: ['13D', '4D', '6S', '8C', '8D', '10H', '12C', '1D', '1H', '8H', '8S', '4S', '10S', '3D', '11C', '4H', '9D', '7S', '1S', '5C', '9S', '6D', '2S', '13S']


Possible Moves:
p2p
source 2C target 4C 3H || 7C 10C 9H 7D 12H 2H
source 4C target 5H || 
source 4C target 5D || 1C 10D 5S 3S 12S
h2p
source 1H target 2C || 11S
source 4S target 5H || 
source 4S target 5D || 1C 10D 5S 3S 12S
source 11C target 13C 12D || 
p2a
h2a
source 1H target _
source 1H target _
source 1H target _
source 1H target _
Move Chosen: source 2C || 11S target 4C 3H || 7C 10C 9H 7D 12H 2H


 Next Round 




===== Aces =====
_ _ _ _ 

===== Player #0 =====
Pile 0: 13C 12D || 
Pile 1: 5H || 
Pile 2: 11S || 
Pile 3: 11H || 2D 11D 13H
Pile 4: 6H || 3C 7H 6C 9C
Pile 5: 5D || 1C 10D 5S 3S 12S
Pile 6: 4C 3H 2C || 7C 10C 9H 7D 12H 2H
---
Hand: ['13D', '4D', '6S', '8C', '8D', '10H', '12C', '1D', '1H', '8H', '8S', '4S', '10S', '3D', '11C', '4H', '9D', '7S', '1S', '5C', '9S', '6D', '2S', '13S']


Possible Moves:
p2p
source 11S target 13C 12D || 
source 4C target 5H || 
source 4C target 5D || 1C 10D 5S 3S 12S
h2p
source 10H target 11S || 
source 1H target 4C 3H 2C || 7C 10C 9H 7D 12H 2H
source 4S target 5H || 
source 4S target 5D || 1C 10D 5S 3S 12S
source 11C target 13C 12D || 
p2a
h2a
source 1H target _
source 1H target _
source 1H target _
source 1H target _
Move Chosen: source 11S ||  target 13C 12D || 


 Next Round 




===== Aces =====
_ _ _ _ 

===== Player #0 =====
Pile 0: 13C 12D 11S || 
Pile 1: 5H || 
Pile 2:  || 
Pile 3: 11H || 2D 11D 13H
Pile 4: 6H || 3C 7H 6C 9C
Pile 5: 5D || 1C 10D 5S 3S 12S
Pile 6: 4C 3H 2C || 7C 10C 9H 7D 12H 2H
---
Hand: ['13D', '4D', '6S', '8C', '8D', '10H', '12C', '1D', '1H', '8H', '8S', '4S', '10S', '3D', '11C', '4H', '9D', '7S', '1S', '5C', '9S', '6D', '2S', '13S']


Possible Moves:


IndexError: list index out of range

In [197]:
x = list(range(10))
np.random.default_rng(1).shuffle(x)
print(x)
x.remove(1)
print(x)

[8, 4, 7, 0, 1, 2, 5, 9, 6, 3]
[8, 4, 7, 0, 2, 5, 9, 6, 3]


Initial Game:
===== Aces =====
_ _ _ _ 

===== Player #0 =====
Pile 0: 7S || 
Pile 1: 2C || 8S
Pile 2: 4S || 9S 5H
Pile 3: 6S || 6C 8C 10S
Pile 4: 1S || 4C 7H 3D 7D
Pile 5: 13C || 10D 1H 6H 2S 13D
Pile 6: 8H || 5D 5S 13H 12D 2H 6D
---
Hand: ['9D', '8D', '1D', '3S', '11H', '11S', '12S', '12H', '7C', '11C', '9C', '1C', '4H', '3C', '10H', '2D', '3H', '9H', '11D', '12C', '5C', '4D', '10C', '13S']


p2p
Possible Moves:
card 7S target 8H || 5D 5S 13H 12D 2H 6D
h2p
Possible Moves:
card 1D target 2C || 8S
card 7C target 8H || 5D 5S 13H 12D 2H 6D
p2a
Possible Moves:
card 1S target _
card 1S target _
card 1S target _
card 1S target _
h2a
Possible Moves:
card 1D target _
card 1D target _
card 1D target _
card 1D target _
card 1C target _
card 1C target _
card 1C target _
card 1C target _



In [153]:
x = [1,2,3]
print(x.pop(0))
print(x)

1
[2, 3]


In [23]:
# FROM ChatGPT

# Test the Card class
def test_card():
    # Test valid card creation
    card = Card(5, 'H')
    assert card.val == 5
    assert card.suit == 'H'
    assert card.color == 'Red'
    assert str(card) == '5H'
    
    card = Card(13, 'S')
    assert card.val == 13
    assert card.suit == 'S'
    assert card.color == 'Black'
    assert str(card) == '13S'
    
    # Test invalid card creation
    try:
        Card(0, 'H')
    except ValueError:
        print("ValueError caught as expected for invalid value")
    
    try:
        Card(14, 'H')
    except ValueError:
        print("ValueError caught as expected for invalid value")
    
    try:
        Card(5, 'X')
    except ValueError:
        print("ValueError caught as expected for invalid suit")

# Test the Deck class
def test_deck():
    # Test deck initialization
    deck = Deck()
    assert len(deck) == 52
    assert isinstance(deck.cards, np.ndarray)
    
    # Test deck shuffle
    deck_str_before = str(deck)
    deck = Deck()  # New deck should be shuffled again
    deck_str_after = str(deck)
    assert deck_str_before != deck_str_after  # Check that the decks are shuffled
    
    # Test draw method
    drawn_cards = deck.draw(5)
    assert len(drawn_cards) == 5
    assert len(deck) == 47  # 52 - 5 = 47
    
    # Test draw more cards than available
    drawn_cards = deck.draw(48)
    assert len(drawn_cards) == 47  # Only 47 cards were left
    assert len(deck) == 0
    
    # Test warning when trying to draw from empty deck
    drawn_cards = deck.draw(1)
    assert len(drawn_cards) == 0


test_card()
test_deck()
print("All card & deck tests passed.")

def testPileAdding():
    d = Deck()
    print(d)
    print('---')
    
    p = Pile(d.cards[:5])
    print(p)
    
    while True:
        c = input('Card: ')
        val = c[:-1]
        suit = c[-1]
        if c == 'break':
            break
        try :
            val = int(val)
        except ValueError:
            print(f'enter a valid number! val = {val}')
        print(p.add(Card(val,suit)))
        print(p)
        print('-----')

def testPile():
    p = AcePile([])
    print(p)
    
    # d = Deck()
    # print(d)
    # print('---')
    # p = HandPile(d.cards[:5])
    # print(p)
    
    while True:
        c = input('Card: ')
        if len(c) == 0 or c == 'break':
            break
        val = c[:-1]
        suit = c[-1]
        try :
            val = int(val)
        except ValueError:
            print(f'enter a valid number! val = {val}')
        c = Card(val,suit)
        print(p.canAdd(c))
        if p.canAdd(c):
            p.add(c)
        print(p)
        print('-----')

def testPossibleMoves():
    def showMoves(moves):
        print('Possible Moves:')
        for move in moves:
            print('card',move['card'],'target',move['target'])
    
    while True:
        x = input()
        if x != '':
            break
        g = Game(1)
        # g.acePiles[0].add(Card(1,'H'))
        # g.acePiles[1].add(Card(1,'D'))
        # g.acePiles[1].add(Card(2,'D'))
        # g.acePiles[2].add(Card(1,'C'))
        # g.acePiles[2].add(Card(2,'C'))
        # g.acePiles[2].add(Card(3,'C'))
        # g.acePiles[3].add(Card(1,'S'))
        g.show()    
        # showMoves(g.players[0].possibleMoves_pileToPile())
        # showMoves(g.players[0].possibleMoves_handToPile())
        # showMoves(g.players[0].possibleMoves_pileToAces(g.acePiles))
        # showMoves(g.players[0].possibleMoves_handToAces(g.acePiles))

value must be between 1 and 13 inclusive! You entered val = 0
ValueError caught as expected for invalid value
value must be between 1 and 13 inclusive! You entered val = 14
ValueError caught as expected for invalid value
suit must be H (heart), D (diamond), C (club), or S (spade). You entered suit = X
ValueError caught as expected for invalid suit
WARNING! Trying to draw from empty deck
All tests passed.


# Questions:
#### Single Player
- Probability of winning?
- Probability of winning vs first moment all kings
- Probability of winning vs color or number diversity of starting board
- Optimal strategy is to build down or aces?
- If allowed to cheat on hand & switch to drawing 2s, does that increase probability of winning significantly?
- Avg number of possible moves at any given time?
#### Multiplayer
- Probability of finishing vs number of players?
- Probability of finishing vs variance in skill of players?
- Probability of winning vs handicaps of not being able to reach certain piles?
- Optimal strategies for finishing

# Idea for definins strategies:
# they take in a list of possible moves, then output the recommendation

# Plan:
# There is a Game class w/ a bunch of players on it
# That is where the .play() function is which loops through turns until game is over and tracks aces, winners, etc

# Each player is a class w/ their deck of cards, hand, & board